In [43]:
import numpy as np
import pandas as pd
import os
import pickle


In [44]:
# Idea: Creating a numerical representation for domain models
# by assigning each model an index
# ----------------------------------------------------------------------
# By collecting all domain models into a list and assigning each model
# an index, we create a numerical representation for each domain model.
# This approach allows us to represent categorical data (domain models)
# in a numerical format, which is essential for many machine learning
# algorithms. It reduces the dimensionality of the feature space compared
# to one-hot encoding, while preserving the inherent ordering or relationship
# between domain models. Additionally, numeric indices are more memory-efficient
# and easier to interpret than one-hot encoded vectors. However, it's crucial
# to ensure consistent index assignment across different datasets and experiments
# to maintain the integrity of our data preprocessing pipeline.


In [45]:
DATA_DIR = "../data"
all_models_sorted_file = os.path.join(DATA_DIR, 'CD_models_curated_sorted')

print(all_models_sorted_file)

all_domain_models = []
with open(all_models_sorted_file, 'r') as f:
    all_domain_models = [d.strip() for d in f.readlines()]

print(len(all_domain_models))
print(all_domain_models[:10])

domain_model_indices = {value: index + 1 for index, value in enumerate(all_domain_models)}

with open(os.path.join(DATA_DIR, 'CD_models_indices.pickle'), 'wb') as f:
    pickle.dump(domain_model_indices, f)
    print('Saved as pickle file')


../data/CD_models_curated_sorted
28055
['CHL00001', 'CHL00002', 'CHL00004', 'CHL00005', 'CHL00008', 'CHL00009', 'CHL00011', 'CHL00012', 'CHL00013', 'CHL00014']
Saved as pickle file


In [46]:
# Encode Superfamily Strings
# ----------------------------------------------------------------------

superfamily_strings_file = os.path.join(DATA_DIR, 'super_families_curated_sorted')

all_superfamily_strings = []

with open(superfamily_strings_file, 'r') as f:
    all_superfamily_strings = [d.strip() for d in f.readlines()]

print(len(all_superfamily_strings))
print(all_superfamily_strings[:10])

superfamily_string_indices = {value: index + 1 for index, value in enumerate(all_superfamily_strings)}

with open(os.path.join(DATA_DIR, 'super_families_indices.pickle'), 'wb') as f:
    pickle.dump(domain_model_indices, f)
    print('Saved as pickle file')



13883
['/NonD', '1-cysPrx_C', '10', '14-3-3', '16S_RimM', '17', '18S_RNA_Rcl1p', '2-oxoacid_dh', '2-oxogl_dehyd_N', '2-ph_phosp']
Saved as pickle file


In [50]:


# Specify the file path of the CSV file
sparcle_data_file_path = os.path.join(DATA_DIR, 'SPARCLE_IDS_curated.csv')

# Specify the columns you want to read from the CSV file
columns_to_read = ['ArchId', 'CurName', 'SpecificArch', 'superfamilyarch'] 

# Read specific columns of the CSV file into a DataFrame
df = pd.read_csv(sparcle_data_file_path, usecols=columns_to_read)

# Display the DataFrame
df.head(20)


,ArchId,CurName,SpecificArch,superfamilyarch
0,3,FtsZ/tubulin family protein,NaN,Tubulin_FtsZ_Cetz-like
1,16,TCP-1/cpn60 chaperonin family protein,NaN,chaperonin_like
2,2,envelope glycoprotein 120,NaN,GP120
3,30,ABC transporter permease,NaN,TM_PBP2
4,21,homeobox domain-containing protein,NaN,homeodomain
5,37,surface antigen,NaN,vMSA
6,76,PTS sugar transporter subunit IIA,NaN,PTS_IIA_fru
7,7,protein Nef,NaN,F-protein
8,13,rhodanese-like domain-containing protein,NaN,RHOD
9,22,thiamine pyrophosphate-dependent enzyme,NaN,TPP_enzymes


In [52]:
# This code converts domain-based architecture strings (from a DataFrame's 'SpecificArch' column) into fixed-length numerical representations. 
# It assumes the following:
#   * Each architecture string consists of space-separated domain names.
#   * A dictionary 'domain_model_indices' exists, mapping domain names to numerical indices. 
#   * The output is a list of NumPy arrays, where each array represents an architecture with domain indices at the beginning, padded with zeros if needed.
#   * The code also tracks the maximum length of any architecture encountered. 


specificArch_converted = []
max_len = 0
for dms in df['SpecificArch']:
    arr = np.zeros(31)
    if not pd.isna(dms):
        a = [domain_model_indices.get(m, 0) for m in dms.split(' ')]
        if len(a) > max_len:
            max_len = len(a)
        arr[:len(a)] = a
        
    specificArch_converted.append(arr)

print(max_len)
print(specificArch_converted[:10])

20
[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0

In [54]:

superfamilyArch_converted = []

for sfs in df['superfamilyarch']:
    if pd.isna(sfs):
        superfamilyArch_converted.append(np.array([0]))
    else:
        superfamilyArch_converted.append(np.array([superfamily_string_indices.get(sfs, 0)]))

print(superfamilyArch_converted[:10])
    

[array([11992]), array([12921]), array([5499]), array([11617]), array([13188]), array([13760]), array([9667]), array([5031]), array([10433]), array([11640])]


In [55]:
df['SpecificArch'] = specificArch_converted
df['superfamilyarch'] = superfamilyArch_converted

df.head(10)

,ArchId,CurName,SpecificArch,superfamilyarch
0,3,FtsZ/tubulin family protein,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[11992]
1,16,TCP-1/cpn60 chaperonin family protein,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[12921]
2,2,envelope glycoprotein 120,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[5499]
3,30,ABC transporter permease,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[11617]
4,21,homeobox domain-containing protein,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[13188]
5,37,surface antigen,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[13760]
6,76,PTS sugar transporter subunit IIA,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[9667]
7,7,protein Nef,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[5031]
8,13,rhodanese-like domain-containing protein,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[10433]
9,22,thiamine pyrophosphate-dependent enzyme,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[11640]
